In [74]:
from py2neo import Graph,RelationshipMatcher,NodeMatcher
from json import dumps

In [4]:
graph = Graph("bolt://127.0.0.1:7687",username='neo4j',password='zhan1216')

In [5]:
re_matcher = RelationshipMatcher(graph)
no_matcher = NodeMatcher(graph)

In [95]:
def direct_search(cls = "Mark Cup"):
    # gather object
    object_node = dict(graph.run("MATCH (n:Object {name:\"%s\"}) return n"%(cls)).next().values()[0])
    all_nodes = graph.run("MATCH (n:Object {name:\"%s\"})-[r:Assembled_Of]->(m) return m"%(cls))
    
    # gather subparts, if None, add object's self as a subpart
    subparts = []
    while all_nodes.forward():
        subparts.append(dict(all_nodes.current.values()[0]))
    if len(subparts)==0:
        subparts.append(object_node)
    subparts = {i['name']:i['description'] for i in subparts}
    
    # gather shapes
    for part_name in subparts:
        description = subparts[part_name]
        subparts[part_name] = {"description":description,"shapes":[]}
        formed_shapes = graph.run("MATCH (n:Object {name:\"%s\"})-[r:Shape_Of]->(m) return m"%(part_name))
        shapes = []
        while formed_shapes.forward():
            shapes.append(dict(formed_shapes.current.values()[0]))
        subparts[part_name]["shapes"] = {i['name']:{"description":i['description'],"grasps":[]} for i in shapes}
    
    # gather grasp types
    all_available_grasps = {}
    for part_name in subparts:
        for shape_name in subparts[part_name]["shapes"]:
            grasp_choices = graph.run("MATCH (n:Shape {name:\"%s\"})-[r:CanApply]->(m) return m"%(shape_name))
            grasps = []
            while grasp_choices.forward():
                grasps.append(dict(grasp_choices.current.values()[0]))
            subparts[part_name]["shapes"][shape_name]['grasps'] = {i['name']:{"description":i['description']} for i in grasps}
            for grasp_type in subparts[part_name]["shapes"][shape_name]['grasps']:
                if grasp_type not in all_available_grasps:
                    all_available_grasps[grasp_type] = 0
                all_available_grasps[grasp_type] += 1
    return subparts,all_available_grasps

In [98]:
gathered_info, gathered_grasp_types = direct_search("Mark Cup")
print("grasp*Shape_cnts: ",dumps(gathered_grasp_types,indent='  '))
print()
print("tree display of info gathered: ",dumps(gathered_info,indent='  '))

grasp*Shape_cnts:  {
  "Hold Grasping": 2,
  "Attract Grasping": 1,
  "Pinch Grasping": 1
}

tree display of info gathered:  {
  "Cup Handle": {
    "description": "the handle of the cup",
    "shapes": {
      "Flared Handle": {
        "description": "handles that often occures on a cup",
        "grasps": {
          "Hold Grasping": {
            "description": "Grasping by Holding the target"
          }
        }
      }
    }
  },
  "Wall of Cup": {
    "description": "the part of the cup without the lid and the handle",
    "shapes": {
      "Cylinder": {
        "description": "Cylinder Shape",
        "grasps": {
          "Attract Grasping": {
            "description": "Grasping by Sucking/Attracting the target"
          },
          "Hold Grasping": {
            "description": "Grasping by Holding the target"
          },
          "Pinch Grasping": {
            "description": "Grasping by Pinching the target"
          }
        }
      }
    }
  }
}
